In [1]:
from transformers import AutoTokenizer
from transformers.models.bert import BertTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

/home/marcus/anaconda3/envs/sd/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-11-26 10:52:20.062544: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
captions = ["fuck", "you ass"]
test = tokenizer(
    "little",
    return_tensors="pt",
    truncation=True,
    padding="max_length",
    max_length=40,
)

from itertools import chain

lab_tokens = test['input_ids']
#lab_tokens = [token[1:-1] for token in lab_tokens]
#lab_tokens = list(chain(*lab_tokens))

print(lab_tokens)

tensor([[ 101, 2210,  102,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0]])


In [19]:
print(tokenizer.get_vocab())
idx2token = {v : k for k, v in tokenizer.get_vocab().items()}

ids = test['input_ids'].numpy().tolist()
words = [[idx2token[id] for id in idxs] for idxs in ids]
print(words)

{'delegation': 10656, 'error': 7561, 'verona': 20197, '##log': 21197, 'severely': 8949, 'oakland': 9182, '##ﬁ': 30510, 'located': 2284, '##iation': 18963, '##ciency': 29125, 'snoop': 29044, 'enduring': 16762, 'hotels': 9275, 'chain': 4677, '##neil': 27276, 'cleanup': 27686, 'yamamoto': 28318, 'activism': 16841, 'we': 2057, 'pinch': 18392, '##lord': 19980, 'transitioned': 23946, '##jure': 25243, 'naming': 10324, '##rricular': 21231, 'homeless': 11573, '1914': 4554, 'liszt': 26273, 'hacking': 23707, '[unused344]': 349, 'mosque': 8806, 'kingdoms': 12028, 'stench': 21555, 'غ': 1289, '##ungen': 23239, 'resist': 9507, 'scowl': 19981, '38th': 22051, 'nueva': 24536, 'almighty': 26668, '##ures': 14900, 'gnome': 25781, 'eagle': 6755, 'pcs': 27019, 'sponsorship': 12026, '##yon': 14001, 'casimir': 24701, 'operating': 4082, '##sus': 13203, '[unused241]': 246, 'fury': 8111, 'availability': 11343, 'quantity': 11712, 'age': 2287, 'utilizing': 16911, 'poor': 3532, 'denied': 6380, '[unused412]': 417, 'i

In [7]:
set([1, 2, 3]) in set([1, 2, 3, 4])

False

In [4]:
import numpy as np
import torch

def get_caption(tokenizer, idx2label, targets=None, 
                prefix=None, suffix=None, max_length=197):
    '''
    Convert (one hot) targets to tokenized captions, 
    meanwhile convert targets to trainable format
    example: [1 0 0 1 0 0] -> ([0 0 1 0 1], [4342, 32, 23, 43, 454]) # maybe \"Detect: cat, dog\"
    Note that the lenght of the returned targets are fixed to \"max_length\"
    '''
    caption = ",".join([x for x in idx2label.values()])

    if prefix:
        caption = prefix + '[SEP]' + caption
    if suffix:
        caption = caption + '[SEP]' + suffix
    
    tokens = tokenizer(
        caption,
        return_tensors="pt",
        truncation=True,
        padding="max_length",
        max_length=max_length,
    )
    one_hot = None

    if targets is not None:
        bz = len(targets)
        tokens = tokenizer(
            [caption] * bz,
            return_tensors="pt",
            truncation=True,
            padding="max_length",
            max_length=max_length,
        )
    
        if isinstance(targets, np.ndarray):
            targets = targets.tolist()
        if isinstance(targets, torch.Tensor):
            targets = targets.detach().cpu().numpy().tolist()

        one_hot = np.zeros((len(targets), max_length))

        for id, target in enumerate(targets):
            labels = [idx2label[i] for i, c in enumerate(target) if c == 1]
            lab_tokens = tokenizer(labels,
                                   truncation=True,)['input_ids']
            lab_tokens = [token[1:-1] for token in lab_tokens]
            lab_tokens = list(chain(*lab_tokens))

            for j, token in enumerate(tokens['input_ids'].numpy()[0, :]):
                one_hot[id, j] = int(token in lab_tokens)
        
    return (tokens, one_hot)


idx2label = {0: "love", 1: "fuck", 2: "you", 3: "ass", 4: "hole"}
targets = [[0, 1, 0, 1], [1, 0, 1, 0]]

res = get_caption(tokenizer, idx2label, targets, "Detect: ", "Desription: flower")

print(res[0])

[[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
  0. 0. 0. 0. 0. 0. 